# Groq API - Memory Copy Tutorial
The following is a very simple tutorial to show copying the contents of one memory location to another memory location.

By the end of this tutorial you should be familiar with the following concepts:
* Different Tensor Types in GroqChip
  * Input tensors
  * Output tensors
  * Memory tensors
  * Stream tensors
* Streams and Stream Groups

It is expected that you have finished reading the Memory Copy section of the Groq API Tutorial Guide prior to going through this tutorial.


## Build Your Program
Begin by importing the Groq API python package and NumPy package. 

In [ ]:
import groq.api as g
from groq.runner import tsp
import numpy as np

print("Python packages imported successfully")

Next, we'll create the input data that we want to create a copy of and store it in memory. Since we know the data at compile time, we'll use `g.from_data()`. If we didn't know the data at compile time, we would use g.input_tensor() instead and provide the data as an input at runtime. This will be demonstrated in the Adding Tensors Tutorial. 

In [ ]:
# Create our input data upfront
t1_data = np.random.rand(32, 32).astype(np.float16) # Random data

# Assign the input data that will be copied
data_mt = g.from_data(data=t1_data)

To create a copy, we'll need to read the data from memory. By convention (and best practice), if the tensor is a memory tensor, we append `_mt` to the variable name. Therefore, since the data is written into memory, we've called it `data_mt`. For streaming tensors, we append `_st`.

The `read` needs to have a stream provided and it is recommended to specify which stream to use. In the following, we're specifying a StreamGroup of 2 streams (you need two streams to represent FP16) and we're going to place it at Index 0. 

Note that `time=0` is included. In the Adding Tensors Tutorial, we discuss time in more depth, however, for right now know that at least one instruction needs to have time assigned. Since `read` is first in our program, we'll set it to 0.

In [ ]:
# Read from memory
data_st = data_mt.read(streams=g.SG2[0], time=0)

With the data read from memory and assigned to data_st (stream tensor), we can now write it. This will write the data to another location in memory, referenced by `data_copy_mt`. Groq API allocates which location in memory the tensor is placed during the compile phase.

In [ ]:
# Write to memory
data_copy_mt = data_st.write() 

## Compilation and Running on Hardware
We are now ready to compile our program. This will create the binary code to be loaded onto the GroqChip. We use the IO Package (IOP) that's created from the compile() step to program the GroqChip.

In [ ]:
#Compile - compilation is run on the host.
iop_file = g.compile(base_name="memcopy", result_tensor=data_copy_mt)

#Program the GroqChip - host sends binary code to GroqChip 
program = tsp.create_tsp_runner(iop_file)

In [ ]:
# Call the program - runtime. Note that we do not need to provide any runtime inputs to the program since the data we copied was preloaded. result is an output tensor from the GroqChip back to the host. 
result = program()

### Confirm Copy
Let's check that the copy of the data is the same as the input. 

In [ ]:
print("Input matches output...")
print((t1_data==result['<ct>.read.write']).all())
print("Input\n",t1_data, "\nOutput\n", result["<ct>.read.write"])


### GroqView (OPTIONAL)
GroqView can be used to view the instructions of your program in the GroqChip. When you click on an instruction, you can get the name of the Tensor API level operation. Note: it is expected that you are familiar with GroqView for the following section of this tutorial. See the GroqView User Guide for more details. 

Using the following command, we can create a .json file that can be used to view the program in hardware. This will show:
* what instructions occur
* where on the chip they take place, as well as
* when in time (cycles) each instruction occurs.

To launch the GroqView tool, uncomment and run the following command. Remember, you still need to create a tunnel to the server running GroqView to load in another window.

Note: before proceeding to the next section, you'll want to stop this cell.

In [ ]:
g.write_visualizer_data("mem_copy")
#!groqview mem_copy/visdata.json

In the GroqView tool, you should see that the tensor is copied 2 bytes * 32-way SIMD per cycle over the course of 32 cycles